In [1]:
import pandas as pd
import numpy as np
import math 
from math import factorial
from scipy.stats import norm
import matplotlib.pyplot as plt
def calculate_combinations(n, r):
    return factorial(n) // factorial(r) // factorial(n-r)


In [2]:
#Build probability matrix
def Prob(Maturity,N,So,r,se):
    T = Maturity/N
    u= math.exp( se * math.sqrt(T))
    d= math.exp( - se * math.sqrt(T))
    p = (math.exp(r*T)-d)/(u-d) 
    result = np.zeros((N+1,N+1))
    for i in range(N+1):
        for j in range(i+1):
            result[i,j] = calculate_combinations(i,j)*(1-p)**j * p**(i-j)
    return result

In [3]:
#Build stock price matrix
def St(Maturity,N,So,r,se):
    T=Maturity/N
    u= math.exp( se * math.sqrt(T))
    d= math.exp( - se * math.sqrt(T))
    p = (math.exp(r*T)-d)/(u-d) 
    result = np.zeros((N+1,N+1))
    for i in range(N+1):
        for j in range(i+1):
            result[i,j] = So*(d**j)*(u**(i-j))
    return result

In [4]:
#Define value of Euro option
def Val_call(K, Maturity,N,So,r,se):
    Stockprice = St(Maturity, N,So,r,se)
    Value = np.zeros((N+1,N+1))
    Probability = Prob(Maturity,N,So,r,se)
    for i in range(N+1):
        for j in range(i+1):
            Value[i,j] = max(Stockprice[i,j]-K, 0)
    return Value
def Val_put(K, Maturity,N,So,r,se):
    Stockprice = St(Maturity, N,So,r,se)
    Value = np.zeros((N+1,N+1))
    Probability = Prob(Maturity,N,So,r,se)
    for i in range(N+1):
        for j in range(i+1):
            Value[i,j] = max(-Stockprice[i,j]+K, 0)
    return Value

# Euro option pricing

In [5]:
def Eu_call(K,Maturity,N,So,r,se):
    result = np.dot(Prob(Maturity,N,So,r,se)[N,:],Val_call(K,Maturity,N,So,r,se)[N,:])*math.exp(-r*Maturity)
    return result
def Eu_put(K,Maturity,N,So,r,se):
    result = np.dot(Prob(Maturity,N,So,r,se)[N,:],Val_put(K,Maturity,N,So,r,se)[N,:])*math.exp(-r*Maturity)
    return result

# American Option Pricing

In [1]:
def Ame_Value_call(K, Maturity,N,So,r,se):
    T = Maturity/N
    u= math.exp( se * math.sqrt(T))
    d= math.exp( - se * math.sqrt(T))
    p = (math.exp(r*T)-d)/(u-d)
    Stockprice = St(Maturity, N,So,r,se)
    Value = np.zeros((N+1,N+1))
    Eu_Val = Val_call(K, Maturity,N,So,r,se)
    Probability = Prob(Maturity,N,So,r,se)
    for i in range(N+1):
        Value[N,i] = max(Stockprice[N,i]-K, 0)
    for i in range(N):
        for j in range(N-1):
            Value[N-1-i,j]=max(Eu_Val[N-1-i,j], np.exp(-r*T/N)*(Value[N-i,j]*p+Value[N-i,j+1]*(1-p)))
    result = Value[0,0]
    return result
def Ame_Value_put(K, Maturity,N,So,r,se):
    T = Maturity/N
    u= math.exp( se * math.sqrt(T))
    d= math.exp( - se * math.sqrt(T))
    p = (math.exp(r*T)-d)/(u-d)
    Stockprice = St(Maturity, N,So,r,se)
    Eu_Val = Val_put(K, Maturity,N,So,r,se)
    Value = np.zeros((N+1,N+1))
    Probability = Prob(Maturity,N,So,r,se)
    for i in range(N+1):
        Value[N,i] = max(-Stockprice[N,i]+K, 0)
    for i in range(N):
        for j in range(N-i):
            Value[N-1-i,j]=max(Eu_Val[N-1-i,j], np.exp(-r*T/N)*(Value[N-i,j]*(p)+Value[N-i,j+1]*(1-p)))
    result = Value[0,0]
    return result
